In [1]:
import time

import numpy as np

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Softmax
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Accuracy, Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau

from HW_base import evaluate_build, focus_build
from HW_keras import hw_layer

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
y_train = (y_train==0).astype(np.float32)
y_test = (y_test==0).astype(np.float32)

In [4]:
# y_train = np.concatenate((y_train==0, y_train != 0), axis=-1).astype(np.float32)
# y_test = np.concatenate((y_test==0, y_test != 0), axis=-1).astype(np.float32)

In [5]:
x_min = np.min(x_train, axis=(1, 2), keepdims=True)
x_max = np.max(x_train, axis=(1, 2), keepdims=True)
x_train = (x_train - x_min) / (x_max - x_min)

In [6]:
x_min = np.min(x_test, axis=(1, 2), keepdims=True)
x_max = np.max(x_test, axis=(1, 2), keepdims=True)
x_test = (x_test - x_min) / (x_max - x_min)

In [7]:
evaluate_list = [evaluate_build(x_test[..., i], 8) for i in range(x_test.shape[-1])]
evaluate_focus_list = [focus_build(evaluate, 0.8) for evaluate in evaluate_list]

evaluate_num:   8,focus:0.8000: 8it [00:00, 7996.77it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, 8004.40it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, 7998.67it/s]


In [8]:
x = Input(shape=((32, 32, 3)))
y = hw_layer(evaluate_focus_list)(x)

y = Conv2D(24, 3, 1, 'same')(y)
y = Conv2D(24, 1, 1, 'valid')(y)
y = MaxPooling2D(2, 2)(y)
y = Softmax(axis=-1)(y)

y = Conv2D(24, 3, 1, 'same')(y)
y = Conv2D(24, 1, 1, 'valid')(y)
y = MaxPooling2D(2, 2)(y)
y = Softmax(axis=-1)(y)

y = Conv2D(24, 3, 1, 'same')(y)
y = Conv2D(24, 1, 1, 'valid')(y)
y = MaxPooling2D(2, 2)(y)
y = Softmax(axis=-1)(y)

y = Flatten()(y)
y = Dense(64)(y)
y = Dense(1)(y)
y = Activation('sigmoid')(y)

model = Model(x, y)
model.summary()

model.compile(loss=Huber(), optimizer=Adam(learning_rate=0.001), metrics=[AUC(), Precision(thresholds=0.5), Recall(thresholds=0.5)])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
hw_layer (hw_layer)          (None, 32, 32, 24)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 24)        5208      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 24)        600       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 24)        0         
_________________________________________________________________
softmax (Softmax)            (None, 16, 16, 24)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 24)        5208  

In [ ]:
model_name = time.strftime('CIFAR10_KERAS_HWNET%Y%m%d%H%M%S')

callback_list = [
    ReduceLROnPlateau(monitor='loss', factor=0.7,  patience=8, min_lr=1e-7, verbose=True),
    EarlyStopping(monitor='loss', patience=32),
    ModelCheckpoint('models/%s_best.h5'%model_name, monitor='loss', save_best_only=True, verbose=False),
    TensorBoard(log_dir='./Log/%s'%model_name)
]

model.fit(x_train, y_train, epochs=200, batch_size=100, validation_data=(x_test, y_test), callbacks=callback_list)

Epoch 1/200
500/500 [==============================] - 12s 16ms/step - loss: 0.0434 - auc: 0.6249 - precision: 0.5657 - recall: 0.0594 - val_loss: 0.0351 - val_auc: 0.8138 - val_precision: 0.7657 - val_recall: 0.1830
Epoch 2/200
500/500 [==============================] - 8s 16ms/step - loss: 0.0355 - auc: 0.8056 - precision: 0.6726 - recall: 0.2346 - val_loss: 0.0339 - val_auc: 0.8262 - val_precision: 0.7023 - val_recall: 0.2760
Epoch 3/200
500/500 [==============================] - 8s 16ms/step - loss: 0.0345 - auc: 0.8184 - precision: 0.7014 - recall: 0.2518 - val_loss: 0.0340 - val_auc: 0.8342 - val_precision: 0.6155 - val_recall: 0.3570
Epoch 4/200
500/500 [==============================] - 8s 16ms/step - loss: 0.0336 - auc: 0.8249 - precision: 0.7058 - recall: 0.2720 - val_loss: 0.0328 - val_auc: 0.8432 - val_precision: 0.6775 - val_recall: 0.3760
Epoch 5/200
500/500 [==============================] - 8s 17ms/step - loss: 0.0327 - auc: 0.8315 - precision: 0.7220 - recall: 0.2940 -